<a href="https://colab.research.google.com/github/kumar-abhishek/handson-ml2/blob/master/new_ecpe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==2.0.0


     |████████████████████████████████| 86.3MB 37kB/s 
     |████████████████████████████████| 450kB 33.5MB/s 
     |████████████████████████████████| 3.8MB 40.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=87fac7e83840a4832552e18b4e33c63cf057109e39522c46e64e8735aca8f369
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation

In [0]:
!pip install tensorflow-probability==0.8.0

     |████████████████████████████████| 2.5MB 4.6MB/s 
ERROR: gym 0.17.1 has requirement cloudpickle<1.4.0,>=1.2.0, but you'll have cloudpickle 1.1.1 which is incompatible.
  Found existing installation: cloudpickle 1.3.0
    Uninstalling cloudpickle-1.3.0:
      Successfully uninstalled cloudpickle-1.3.0
  Found existing installation: tensorflow-probability 0.10.0rc0
    Uninstalling tensorflow-probability-0.10.0rc0:
      Successfully uninstalled tensorflow-probability-0.10.0rc0


In [0]:
%tensorflow_version 2.x
%tensorflow_version



Currently selected TF version: 2.x
Available versions:
* 1.x
* 2.x


In [0]:

import unicodedata
import re
import numpy as np
import os
import io
import time
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
emotion_seeds = set(["ashamed", "delighted", "pleased", "concerned", "delight", "happy", "embarrassed", "furious", "nervous", 
                     "miffed", "angry", "mad", "anger", "excitement", "horror", "resentful", "astonished", "revulsion", 
                     "frightened", "cross", "sad", "down", "astonishment", "miserable", "worried", "sorrow", "overjoyed",
                     "dismay", "grief", "annoyance", "alarmed", "astounded", "anguish", "despair", "infuriated", 
                     "embarrassment", "peeved", "amused", "disgruntled", "indignant", "thrilled", "anxious", "excited",
                     "exasperation", "petrified", "heartbroken", "saddened", "depressed", "dismayed", "frustrated", "fedup", "livid",
                     "revulsion", "bewildered", "flabbergasted", "happier", "ecstatic", "elation", "exhilarated", "exhilaration",
                     "glee", "gleeful", "crestfallen", "sadness", "amusement", "dejected", "desolate", "despondency", "horrors",
                     "agitated", "disquiet", "horrified", "exasperated", "irked", "disgruntlement", "sickened", "revolted",
                     "devastated", "heartbreak", "inconsolable", "bewilderment", "nonplussed", "puzzlement", "disquieted",
                     "glum", "downcast", "griefstricken", "startled", "disgusted"])

In [0]:
emotion_seeds

{'agitated',
 'alarmed',
 'amused',
 'amusement',
 'anger',
 'angry',
 'anguish',
 'annoyance',
 'anxious',
 'ashamed',
 'astonished',
 'astonishment',
 'astounded',
 'bewildered',
 'bewilderment',
 'concerned',
 'crestfallen',
 'cross',
 'dejected',
 'delight',
 'delighted',
 'depressed',
 'desolate',
 'despair',
 'despondency',
 'devastated',
 'disgruntled',
 'disgruntlement',
 'disgusted',
 'dismay',
 'dismayed',
 'disquiet',
 'disquieted',
 'down',
 'downcast',
 'ecstatic',
 'elation',
 'embarrassed',
 'embarrassment',
 'exasperated',
 'exasperation',
 'excited',
 'excitement',
 'exhilarated',
 'exhilaration',
 'fedup',
 'flabbergasted',
 'frightened',
 'frustrated',
 'furious',
 'glee',
 'gleeful',
 'glum',
 'grief',
 'griefstricken',
 'happier',
 'happy',
 'heartbreak',
 'heartbroken',
 'horrified',
 'horror',
 'horrors',
 'inconsolable',
 'indignant',
 'infuriated',
 'irked',
 'livid',
 'mad',
 'miffed',
 'miserable',
 'nervous',
 'nonplussed',
 'overjoyed',
 'peeved',
 'petrifi

In [0]:
# input is sentence, output is emotion cause pairs
# Determine clauses by splitting on punctuation.

# preprocess

# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

def remove_nonascii(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()
  return w


def preprocess_sentence(w):
  w = remove_nonascii(w)  
  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [0]:
def extract_cause(text):
  cur_cause=''
  try:
    cur_cause = re.findall('<cause>(.*?)<\\\cause>', text)[0]
    # Remove tags from line
    text=re.sub('<cause>', '', text)
    text=re.sub('<\\\cause>', '', text)
  except:
    pass
  #print('here:', text)
  return (cur_cause, text)

In [0]:
def clean_filter_clauses(all_clauses):
  cause = ''
  clauses=[]
  for clause in all_clauses:
    e_cause, e_text = extract_cause(clause)
    if e_cause!='':
      cause = remove_nonascii(e_cause)
    clauses.append(remove_nonascii(e_text))
  return cause, clauses



In [0]:

path_to_file = "data.txt"  
# 1. Remove any accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [document, emotion, cause, clauses list]
document=[]
emotion=[]
cause=[]
clause=[]
clause_global=[]
emotion_label=[]
cause_label=[]
def create_dataset(path, num_examples):
  document.clear()
  emotion.clear()
  cause.clear()
  clause.clear()
  clause_global.clear()
  emotion_label.clear()
  cause_label.clear()
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
  for i, line in enumerate(lines[:num_examples]):
    cur_emotion = re.findall('<(.*?)>', line)[0]
    # removing emotion tag in document
    text_without_emotion=line[2+len(cur_emotion):len(line)-len(cur_emotion)-3]
    #document.append(text_without_emotion)
    emotion.append(cur_emotion)

    # Determine clauses by splitting on punctuation.
    all_clauses = re.split("[.,!;:\"]+", text_without_emotion)
    filter_cause, filter_clauses = clean_filter_clauses(all_clauses)
    cause.append(filter_cause)
    clause.append([filter_clauses])
    doc = extract_cause(text_without_emotion)[1]
    # clean up document
    clean_doc = preprocess_sentence(doc)
    document.append(clean_doc)

    clause_global.extend(filter_clauses)
    for fclause in filter_clauses:
      if fclause==filter_cause:
        cause_label.append(1)
      else:
        cause_label.append(0)

      has_emotion_label=False
      for eseed in emotion_seeds:
        if eseed in fclause:
          has_emotion_label=True
          emotion_label.append(eseed)
          break
      if has_emotion_label==False:
        emotion_label.append('')
        
  return [document, emotion, cause, clause, clause_global, cause_label, emotion_label]

In [0]:
document, emotion, cause, clause_list, clause_global, cause_label, emotion_label = create_dataset(path_to_file, 500)
print('number of documents: ', len(document),'\n')
for i in range(5):
  # print(document[i])
  #print('emotion', emotion[i])
  # print(cause[i])
  # print(clause_list[i])
  # print('\n')
  print('clause: ', clause_global[i])
  print('cause label: ', cause_label[i])
  print('emotion label: ', emotion_label[i])
  print('\n--------\n')

number of documents:  500 

clause:  i suppose i am happy
cause label:  0
emotion label:  happy

--------

clause:  being so tiny
cause label:  1
emotion label:  

--------

clause:  it means i am able to surprise people with what is generally seen as my confident and outgoing personality
cause label:  0
emotion label:  

--------

clause:  
cause label:  0
emotion label:  

--------

clause:  lennox has always truly wanted to fight for the world title and was happy
cause label:  0
emotion label:  happy

--------



In [0]:
# X=document
# y=emotion
# y=np.asarray(y)
# print(y, type(y))

X=clause_global
y_cause=np.asarray(cause_label)
y_emotion=np.asarray(emotion_label)

In [0]:
print(clause_global)
print( type(clause_global))

print('y_cause: ', y_cause, type(y_cause))
print('y_emotion: ', y_emotion)

['i suppose i am happy', 'being so tiny', 'it means i am able to surprise people with what is generally seen as my confident and outgoing personality', '', 'lennox has always truly wanted to fight for the world title and was happy', 'because he was taking the tough route', '', 'he was a professional musician now', 'still sensitive and happy', 'doing something he loved', '', 'holmes is happy', 'because', 'he has the freedom of the house when we are out', '', 'i had problems with tutors trying to encourage me to diversify my work and experiment with other styles', 'but i was quite happy', 'with the direction my work was heading so i stubbornly stuck to it', '', 'these days he is quite happy', 'since', 'he is travelling by trolley', '', 'i m really happy', 'in the group now', '', '', 'i was given a dummy in which to do my preliminary sketches', 'but the dummy was designed so beautifully by amelia edwards that i was really happy', 'with my work', '', 'i must say i was totally happy', 'abou

In [0]:
tokenizer_emotion=keras.preprocessing.text.Tokenizer(num_words=1000, oov_token="xxxxxxx")
tokenizer_emotion.fit_on_texts(emotion_seeds)
print(len(tokenizer_emotion.word_index), len(emotion_seeds), tokenizer_emotion.word_index) 
y_emotion_tokenized=[]
for emotion in y_emotion:
  if emotion in tokenizer_emotion.word_index:
    y_emotion_tokenized.append(tokenizer_emotion.word_index[emotion])
  else:
    y_emotion_tokenized.append(1)
y_emotion_tokenized = np.asarray(y_emotion_tokenized)
print(y_emotion_tokenized)

89 88 {'xxxxxxx': 1, 'exhilarated': 2, 'revulsion': 3, 'infuriated': 4, 'elation': 5, 'worried': 6, 'bewilderment': 7, 'horrors': 8, 'indignant': 9, 'sickened': 10, 'frustrated': 11, 'ecstatic': 12, 'overjoyed': 13, 'ashamed': 14, 'horrified': 15, 'peeved': 16, 'delighted': 17, 'disquieted': 18, 'heartbroken': 19, 'sad': 20, 'thrilled': 21, 'excited': 22, 'dejected': 23, 'astonishment': 24, 'horror': 25, 'sorrow': 26, 'fedup': 27, 'amusement': 28, 'glee': 29, 'downcast': 30, 'nervous': 31, 'revolted': 32, 'grief': 33, 'saddened': 34, 'livid': 35, 'depressed': 36, 'desolate': 37, 'inconsolable': 38, 'angry': 39, 'disgruntlement': 40, 'anger': 41, 'startled': 42, 'anxious': 43, 'nonplussed': 44, 'miserable': 45, 'exhilaration': 46, 'despondency': 47, 'anguish': 48, 'pleased': 49, 'petrified': 50, 'sadness': 51, 'miffed': 52, 'cross': 53, 'astonished': 54, 'gleeful': 55, 'exasperated': 56, 'frightened': 57, 'alarmed': 58, 'bewildered': 59, 'down': 60, 'disgruntled': 61, 'happy': 62, 'asto

In [0]:
tokenizer=keras.preprocessing.text.Tokenizer(num_words=1000, oov_token="xxxxxxx")
tokenizer_emotion=keras.preprocessing.text.Tokenizer(num_words=1000, oov_token="xxxxxxx")

tokenizer.fit_on_texts(X)
tokenizer_emotion.fit_on_texts(emotion_seeds)
X_dict=tokenizer.word_index

X_seq=tokenizer.texts_to_sequences(X)
X_padded_seq=pad_sequences(X_seq,padding='post',maxlen=30) 
print(X_padded_seq[:3], X_padded_seq.shape, type(X_padded_seq))
print(X_padded_seq.shape)


[[  8 450   8  55  45   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0]
 [ 39  29 826   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0]
 [ 19 451   8  55 246   3 452 112  13  56  27 453 308  30  42 827   6 828
  829   0   0   0   0   0   0   0   0   0   0   0]] (2020, 30) <class 'numpy.ndarray'>
(2020, 30)


In [0]:
import tensorflow_hub as hub
from keras.models import Model, Sequential

clauses_model = keras.layers.Input(shape=X_padded_seq.shape[1:], name='clause_input')
clauses_embeddings = keras.layers.Embedding(input_length=30,input_dim=1000,output_dim=50) (clauses_model)
output1 = keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True))(clauses_embeddings)

attention_output = keras.layers.Attention()([output1, output1, output1]) # no idea why we use output3, 3 times TODO!!!

output_emotion = keras.layers.Bidirectional(keras.layers.LSTM(64))(attention_output)
output_emotion_dense1 = keras.layers.Dense(128, activation="relu") (output_emotion)
output_emotion_dense2 = keras.layers.Dense(len(tokenizer_emotion.word_index), activation='softmax', name='emotion_output') (output_emotion_dense1)


output_cause = keras.layers.Bidirectional(keras.layers.LSTM(64))(attention_output)
output_cause_dense1 = keras.layers.Dense(128, activation="relu") (output_cause)
#output_cause_dense2 = keras.layers.Dense(len(tokenizer.word_index), activation='softmax', name='cause_output') (output_cause_dense1)
output_cause_dense2 = keras.layers.Dense(1, activation='sigmoid', name='cause_output') (output_cause_dense1)

model = keras.Model(inputs=clauses_model, outputs=[output_emotion_dense2, output_cause_dense2])
model.compile(loss={'emotion_output': 'sparse_categorical_crossentropy', 'cause_output': 'binary_crossentropy'},
              optimizer="adam", loss_weights=[0.5, 0.5], metrics=['accuracy'])



In [0]:
model.fit({'clause_input': X_padded_seq}, {'emotion_output': y_emotion_tokenized, 'cause_output': y_cause}, epochs=15)

Train on 2020 samples
Epoch 1/15
2020/2020 [==============================] - 4s 2ms/sample - loss: 0.6415 - emotion_output_loss: 1.0248 - cause_output_loss: 0.2578 - emotion_output_accuracy: 0.7535 - cause_output_accuracy: 0.9163
Epoch 2/15
2020/2020 [==============================] - 4s 2ms/sample - loss: 0.5964 - emotion_output_loss: 0.9418 - cause_output_loss: 0.2368 - emotion_output_accuracy: 0.7609 - cause_output_accuracy: 0.9272
Epoch 3/15
2020/2020 [==============================] - 4s 2ms/sample - loss: 0.5462 - emotion_output_loss: 0.8658 - cause_output_loss: 0.2228 - emotion_output_accuracy: 0.7688 - cause_output_accuracy: 0.9332
Epoch 4/15
2020/2020 [==============================] - 4s 2ms/sample - loss: 0.5167 - emotion_output_loss: 0.8156 - cause_output_loss: 0.2132 - emotion_output_accuracy: 0.7861 - cause_output_accuracy: 0.9381
Epoch 5/15
2020/2020 [==============================] - 4s 2ms/sample - loss: 0.4823 - emotion_output_loss: 0.7724 - cause_output_loss: 0.1879

In [0]:
print(model.metrics_names)

print(model.evaluate(X_padded_seq, y_cause, verbose=0))

print(model.evaluate(X_padded_seq, y_emotion, verbose=0))


['loss', 'emotion_output_loss', 'cause_output_loss', 'emotion_output_accuracy', 'cause_output_accuracy']


ValueError: ignored

In [0]:
#print(X_padded_seq)
#print(clause_global[:30])
cause_
print('causes: ', model.predict({'clause_input': X_padded_seq})[1][0:30])
print('-----')
print('emotions: ', model.predict({'clause_input': X_padded_seq})[0][1])

causes:  [[0.00467286]
 [0.992792  ]
 [0.02149299]
 [0.06135821]
 [0.00425297]
 [0.9056627 ]
 [0.06135821]
 [0.04169598]
 [0.00545982]
 [0.9654536 ]
 [0.06135821]
 [0.04261762]
 [0.08833924]
 [0.9931287 ]
 [0.06135821]
 [0.06187195]
 [0.00519952]
 [0.02860957]
 [0.06135821]
 [0.00462639]
 [0.07743061]
 [0.9834853 ]
 [0.06135821]
 [0.00577381]
 [0.05706501]
 [0.06135821]
 [0.06135821]
 [0.01724556]
 [0.00385681]
 [0.99007004]]
-----
emotions:  [2.76657391e-10 9.74585474e-01 3.67879024e-04 2.77448225e-06
 2.81436442e-05 6.73639181e-04 2.37168797e-06 2.15344357e-06
 6.58098907e-06 3.18277071e-10 2.27236596e-05 1.91533282e-11
 1.35762430e-05 6.75020772e-07 4.47698267e-06 1.49005774e-09
 7.46561923e-10 2.88858804e-09 2.57263742e-11 5.14428439e-06
 8.92757812e-10 3.93658041e-07 7.06891683e-07 8.25328425e-06
 3.73397242e-08 4.18882116e-11 5.67624147e-06 1.34727923e-10
 6.57428336e-06 3.84644605e-04 1.60065429e-06 5.04704713e-06
 4.07084963e-06 1.82253341e-08 1.41259657e-10 2.08438111e-09
 8.3

In [0]:
# # test code to test the model prediction
# test_input=[['he was mad'], ['because I am sick']]
# test_X_seq=tokenizer.texts_to_sequences(test_input)
# print(test_X_seq)

# test_X_padded_seq=pad_sequences(test_X_seq,padding='post',maxlen=30)
# print(test_X_padded_seq)

# #output = model.predict_classes(test_X_padded_seq)
# output = model.predict(test_X_padded_seq)
# print(output[0], output[1])
# # for word, index in tokenizer_emotion.word_index.items():
# #   if index==output:
# #     print(word, index, output)
# #     break

In [0]:
#input : emotion * cause


# logistic regresson



#output